# Running C code from Python with `ctypes`

In this notebook we show an example on compiling C code defining an algorithm and then applying it to a numpy array.
The algorithm is very simple: it retrieves the maximum element of a 1D array.

Let's start by writing the algorithm in a file, say `tmp.C`.

Note that the function is defined as an `extern "C"` function. 
This disables C++ name mangling and ease accessing to the compiled code from Python.

In [30]:
print("""
extern "C"
double compute_max (int n, double* data)
{
    int i;
    double max = data[0];
    
    for (i = 1; i < n; ++i)
        if (data[i] > max)
            max = data[i];
    
    return max;
}
""", file=open('tmp.C', 'w'))

Then we compile it with GCC. 
Note that:
 * `-fPIC --shared` are GCC options used to compile the code as a shared object;
 * `-o tmp.so` is the output file path;
 * `tmp.C` is the source.

In [31]:
!gcc -fPIC --shared -o tmp.so tmp.C

## Importing the function with `ctypes`

See [`ctypes`](https://docs.python.org/3/library/ctypes.html) library in Python is designed to enable accessing non-Python libraries from Python.

Here, we are using it to load the `compile_max` function from our `tmp.so` library.

In [33]:
import numpy as np 
import ctypes

## Load the library
library = ctypes.CDLL("./tmp.so")

## Load the function
c_max = library.compute_max

## Define the types of the arguments and return value. 
## Note: np.ctypeslib enable the definition of data types for handling numpy arrays from ctypes.
c_max.argtypes = [ctypes.c_int, np.ctypeslib.ndpointer(dtype=np.float64)]
c_max.restype = ctypes.c_double

We can then use the function as a normal Python function.

In [35]:
array = np.random.normal(0, 2, 100)

m1 = c_max(len(array), array)
m2 = np.max(array)
print (f"({m1} == {m2}) is {m1 == m2}")

(6.019066512638797 == 6.019066512638797) is True


## Compile and load to a random library (to avoid Python caching of your library)

Python loads the library the first time you issue `ctypes.CDLL()` and then to reload the library, you should restart the Python kernel, which is not very effective while prototyping on a notebook.
A workaround is to write on a random file in order to force Python to reload the library.
With time, Python garbage collector will automatically unload the variables.

Here is an example to achieve this.

In [41]:
import random

## Randomize file name
my_lib_path = f"./tmp.{random.randint(0, 0x7FFF)}.so"

## Compile the library with the randomized name 
!gcc -fPIC --shared -o {my_lib_path} tmp.C

## Loads the library reading the randomized name
my_lib = ctypes.CDLL(my_lib_path)